In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
def get_data(batch_size=128, dims=(3800, 4)):
    return np.random.randn(128, *dims), np.random.randint(0, 5, size=batch_size)

def floats_feature(arr):
    return tf.train.Feature(float_list=tf.train.FloatList(value=arr.flatten().tolist()))

def int_feature(i):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[i]))

In [ ]:
features, target = get_data()
output_fname='test.tfrecords'

writer = tf.python_io .TFRecordWriter(output_fname)
for i, (np_features, np_target) in enumerate(zip(features, target)):
    tf_features_map = {
        'ch%i'%ch: floats_feature(x)
        for ch, x in enumerate(np_features)
    }
    tf_features_map['target'] = int_feature(np_target)
    tf_features = tf.train.Features(feature=tf_features_map)
    tf_example = tf.train.Example(features=tf_features)

    writer.write(tf_example.SerializeToString())

writer.close()